# Week 3: Control Theory & Actuation

## Module I: Introduction & Foundations

### Topics Covered

- Drive-by-Wire systems
- Longitudinal Control (PID, Cruise Control)
- Lateral Control (Pure Pursuit, Stanley Method)

---

## Learning Objectives

By the end of this notebook, you will be able to:

1. Understand drive-by-wire architecture and safety requirements
2. Design and tune PID controllers for longitudinal speed control
3. Implement cruise control and adaptive cruise control (ACC)
4. Apply Pure Pursuit algorithm for path following
5. Implement Stanley method for lateral control
6. Compare different control strategies and tune parameters

---

## Setup

Import required libraries for control systems and visualization

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Circle, FancyArrow, Rectangle
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

# Set random seed for reproducibility
np.random.seed(42)

# Plotting configuration
plt.rcParams['figure.figsize'] = (14, 8)
plt.rcParams['font.size'] = 10

print("Libraries loaded successfully!")
print("NumPy version:", np.__version__)

## 1. Drive-by-Wire Systems

**Drive-by-wire** (DbW) is the use of electronic controls to operate vehicle functions traditionally controlled by mechanical linkages. This technology is essential for autonomous vehicles.

### Traditional vs. Drive-by-Wire

| Function | Traditional (Mechanical) | Drive-by-Wire (Electronic) |
|----------|--------------------------|----------------------------|
| **Steering** | Steering column, rack & pinion | Electric motor + sensors |
| **Throttle** | Cable to throttle body | Electronic throttle control (ETC) |
| **Braking** | Hydraulic master cylinder | Electro-hydraulic brake system |
| **Shifting** | Mechanical shifter linkage | Electronic transmission control |

---

### Architecture

```
┌─────────────────────────────────────────────────────────────┐
│                    VEHICLE ECU (Electronic Control Unit)     │
│  ┌──────────────┐  ┌──────────────┐  ┌──────────────┐      │
│  │  Planning &  │→ │   Control    │→ │  Actuation   │      │
│  │  Perception  │  │  Algorithms  │  │   Commands   │      │
│  └──────────────┘  └──────────────┘  └──────────────┘      │
└─────────────────────────────────────────────────────────────┘
         ↓                    ↓                    ↓
┌─────────────┐      ┌─────────────┐      ┌─────────────┐
│  Steering   │      │  Throttle   │      │    Brake    │
│   Motor     │      │   Actuator  │      │   Actuator  │
└─────────────┘      └─────────────┘      └─────────────┘
         ↓                    ↓                    ↓
┌─────────────┐      ┌─────────────┐      ┌─────────────┐
│  Position   │      │   Throttle  │      │   Brake     │
│   Sensors   │      │   Position  │      │   Pressure  │
└─────────────┘      └─────────────┘      └─────────────┘
         ↓                    ↓                    ↓
    (Feedback)           (Feedback)           (Feedback)
```

---

### Key Components

#### 1. **Steer-by-Wire**
- **Electric motor** applies steering torque
- **Angle sensor** measures steering wheel position
- **Torque sensor** measures driver input
- **Redundant systems** for safety

**Advantages**:
- ✅ Removes steering column (more interior space)
- ✅ Variable steering ratio (parking vs. highway)
- ✅ Enable autonomous steering
- ✅ Better crash safety

**Challenges**:
- ❌ Requires redundant power and computing
- ❌ Must provide steering feel (haptic feedback)
- ❌ Regulatory approval needed

---

#### 2. **Throttle-by-Wire (Electronic Throttle Control)**
- **Accelerator pedal sensor** measures driver input
- **ECU** calculates desired throttle opening
- **Throttle actuator** (DC motor) controls throttle plate
- **Throttle position sensor** provides feedback

**Benefits**:
- Smoother throttle response
- Traction control integration
- Cruise control compatibility
- Better fuel economy

---

#### 3. **Brake-by-Wire**
- **Brake pedal sensor** measures driver input
- **Hydraulic modulator** or **electric calipers** apply braking force
- **Wheel speed sensors** for ABS integration
- **Pressure sensors** for feedback control

**Types**:
- **Electro-Hydraulic Brake (EHB)**: Electronic control, hydraulic actuation
- **Electro-Mechanical Brake (EMB)**: Fully electric calipers (future)

---

### Safety Requirements (ISO 26262)

Drive-by-wire systems must meet **ASIL-D** (Automotive Safety Integrity Level D) - the highest safety level:

1. **Redundancy**: Dual or triple redundant sensors and actuators
2. **Fail-Operational**: System continues operating after single failure
3. **Diagnostics**: Continuous self-monitoring and fault detection
4. **Graceful Degradation**: Safe fallback modes
5. **Watchdog Timers**: Detect software/hardware hangs

**Example**: Steer-by-wire might have:
- 2 steering angle sensors
- 2 motor windings
- 2 independent ECUs
- Mechanical backup connection

---

### Control Loop

Drive-by-wire uses **closed-loop control** with feedback:

```
Reference     ┌──────────┐    Command    ┌──────────┐    Actual
Input    +    │          │               │          │    Output
────────>●───>│Controller├──────────────>│ Actuator ├────────>
         │-   │          │               │          │    │
         │    └──────────┘               └──────────┘    │
         │                                               │
         └───────────────────────────────────────────────┘
                          Feedback
```

**PID Controller** is commonly used for low-level actuation control.

---

### Communication: CAN Bus

**Controller Area Network (CAN)** is the standard for automotive communication:
- **Speed**: 500 kbit/s (CAN 2.0) to 5 Mbit/s (CAN-FD)
- **Topology**: Multi-master bus
- **Reliability**: Error detection, arbitration
- **Latency**: 1-10ms typical

**Example CAN Messages**:
- `0x100`: Steering angle command (10 Hz)
- `0x200`: Throttle position command (20 Hz)
- `0x300`: Brake pressure command (50 Hz)
- `0x400`: Wheel speeds (100 Hz)

## 2. Longitudinal Control (Speed Control)

**Longitudinal control** manages the vehicle's forward/backward motion by controlling throttle and brake to achieve desired speed or maintain safe following distance.

### PID Controller Fundamentals

The **PID (Proportional-Integral-Derivative) controller** is the workhorse of industrial control systems. It calculates control output based on error between desired and actual values.

**Control Law**:
$$u(t) = K_p \cdot e(t) + K_i \int_0^t e(\tau) d\tau + K_d \frac{de(t)}{dt}$$

Where:
- **u(t)**: Control output (throttle/brake command)
- **e(t) = v_{target} - v_{actual}**: Speed error
- **K_p**: Proportional gain
- **K_i**: Integral gain
- **K_d**: Derivative gain

---

### Three Control Terms

#### 1️⃣ **Proportional (P) Term**
$$u_P(t) = K_p \cdot e(t)$$

**What it does**: React to current error
- Large error → Large control action
- Small error → Small control action

**Effect**:
- ✅ Fast response
- ❌ Steady-state error (never reaches exact target)
- ❌ Can overshoot

**Example**: If car is 10 m/s below target, apply throttle proportional to 10 m/s

---

#### 2️⃣ **Integral (I) Term**
$$u_I(t) = K_i \int_0^t e(\tau) d\tau$$

**What it does**: Accumulate past errors
- Eliminates steady-state error
- "Remembers" persistent errors

**Effect**:
- ✅ Reaches exact target value
- ❌ Can cause overshoot
- ❌ **Integral windup** if saturated

**Example**: If car consistently 1 m/s too slow, integral builds up and increases throttle

---

#### 3️⃣ **Derivative (D) Term**
$$u_D(t) = K_d \frac{de(t)}{dt}$$

**What it does**: Predict future error based on rate of change
- Damping effect
- Reduces overshoot

**Effect**:
- ✅ Reduces oscillations
- ✅ Faster settling
- ❌ Amplifies measurement noise
- ❌ **Derivative kick** on setpoint changes

**Example**: If error is decreasing rapidly, reduce control action to prevent overshoot

---

### Discrete-Time Implementation

For digital control systems (typical automotive ECU runs at 10-100 Hz):

$$\begin{align}
e(k) &= v_{target}(k) - v_{actual}(k) \\
u_P(k) &= K_p \cdot e(k) \\
u_I(k) &= u_I(k-1) + K_i \cdot e(k) \cdot \Delta t \\
u_D(k) &= K_d \cdot \frac{e(k) - e(k-1)}{\Delta t} \\
u(k) &= u_P(k) + u_I(k) + u_D(k)
\end{align}$$

---

### Anti-Windup for Integral Term

**Problem**: When actuator saturates (e.g., throttle at 100%), integral keeps accumulating.

**Solution**: Clamp integral term

```python
# Anti-windup
integral = integral + Ki * error * dt
integral = np.clip(integral, integral_min, integral_max)
```

---

### Derivative Kick Prevention

**Problem**: Sudden setpoint change causes large derivative spike.

**Solution**: Take derivative of **measurement**, not error

```python
# Instead of: derivative = Kd * (error - prev_error) / dt
# Use:
derivative = -Kd * (measurement - prev_measurement) / dt
```

---

### Tuning PID Gains

#### **Ziegler-Nichols Method**
1. Set Ki = Kd = 0
2. Increase Kp until system oscillates
3. Record Ku (ultimate gain) and Pu (oscillation period)
4. Calculate:
   - Kp = 0.6 × Ku
   - Ki = 2 × Kp / Pu
   - Kd = Kp × Pu / 8

#### **Manual Tuning Guidelines**
1. **Start with P-only**: Increase Kp until fast response with acceptable overshoot
2. **Add I**: Increase Ki to eliminate steady-state error (start small!)
3. **Add D**: Increase Kd to reduce overshoot (optional)

**Typical ranges for cruise control**:
- Kp: 100-500 (N per m/s error)
- Ki: 10-50 (N per m error accumulated)
- Kd: 10-100 (N per m/s² rate)

In [ ]:
# PID Controller Implementation for Cruise Control

class PIDController:
    """
    PID controller for longitudinal speed control.
    """
    
    def __init__(self, Kp, Ki, Kd, dt=0.1, output_limits=None, integral_limits=None):
        """
        Args:
            Kp: Proportional gain
            Ki: Integral gain
            Kd: Derivative gain
            dt: Time step (s)
            output_limits: (min, max) tuple for control output
            integral_limits: (min, max) tuple for integral term (anti-windup)
        """
        self.Kp = Kp
        self.Ki = Ki
        self.Kd = Kd
        self.dt = dt
        
        self.output_limits = output_limits if output_limits else (-np.inf, np.inf)
        self.integral_limits = integral_limits if integral_limits else (-np.inf, np.inf)
        
        # State variables
        self.integral = 0.0
        self.prev_error = 0.0
        self.prev_measurement = 0.0
        
    def update(self, setpoint, measurement, use_derivative_on_measurement=True):
        """
        Calculate PID control output.
        
        Args:
            setpoint: Desired value (target speed)
            measurement: Current value (actual speed)
            use_derivative_on_measurement: If True, take derivative of measurement
                                           (prevents derivative kick)
        
        Returns:
            control_output: PID control signal
        """
        # Calculate error
        error = setpoint - measurement
        
        # Proportional term
        P = self.Kp * error
        
        # Integral term with anti-windup
        self.integral += error * self.dt
        self.integral = np.clip(self.integral, *self.integral_limits)
        I = self.Ki * self.integral
        
        # Derivative term
        if use_derivative_on_measurement:
            # Derivative on measurement (prevents derivative kick)
            D = -self.Kd * (measurement - self.prev_measurement) / self.dt
            self.prev_measurement = measurement
        else:
            # Derivative on error (traditional)
            D = self.Kd * (error - self.prev_error) / self.dt
            self.prev_error = error
        
        # Total control output
        output = P + I + D
        
        # Apply output limits
        output = np.clip(output, *self.output_limits)
        
        return output, P, I, D
    
    def reset(self):
        """Reset controller state."""
        self.integral = 0.0
        self.prev_error = 0.0
        self.prev_measurement = 0.0


# Simple vehicle dynamics model
class SimpleLongitudinalVehicle:
    """Simplified vehicle model for longitudinal dynamics."""
    
    def __init__(self, mass=1500, drag_coef=0.3, rolling_resistance=0.01):
        """
        Args:
            mass: Vehicle mass (kg)
            drag_coef: Aerodynamic drag coefficient
            rolling_resistance: Rolling resistance coefficient
        """
        self.mass = mass
        self.drag = drag_coef
        self.rolling_resistance = rolling_resistance
        self.velocity = 0.0  # m/s
        
    def update(self, force, dt=0.1):
        """
        Update vehicle state given applied force.
        
        Args:
            force: Net force (N) - positive for acceleration, negative for braking
            dt: Time step (s)
        """
        # Resistance forces
        drag_force = self.drag * self.velocity**2
        rolling_force = self.rolling_resistance * self.mass * 9.81
        
        # Net force
        net_force = force - drag_force - rolling_force
        
        # Acceleration (F = ma)
        acceleration = net_force / self.mass
        
        # Update velocity
        self.velocity += acceleration * dt
        self.velocity = max(0, self.velocity)  # No negative velocity
        
        return self.velocity


# Simulation: Cruise Control with Different PID Tunings
dt = 0.1
duration = 30  # seconds
time = np.arange(0, duration, dt)

# Target speed profile (step changes)
target_speed = np.ones_like(time) * 20.0  # 20 m/s
target_speed[int(10/dt):] = 25.0  # Increase to 25 m/s at t=10s

# PID configurations to compare
pid_configs = {
    'P-only (Kp=200)': {'Kp': 200, 'Ki': 0, 'Kd': 0},
    'PI (Kp=200, Ki=20)': {'Kp': 200, 'Ki': 20, 'Kd': 0},
    'PID (Kp=200, Ki=20, Kd=50)': {'Kp': 200, 'Ki': 20, 'Kd': 50},
    'Aggressive PID (Kp=400, Ki=40, Kd=100)': {'Kp': 400, 'Ki': 40, 'Kd': 100}
}

results = {}

for name, params in pid_configs.items():
    # Initialize controller and vehicle
    controller = PIDController(
        Kp=params['Kp'], Ki=params['Ki'], Kd=params['Kd'], dt=dt,
        output_limits=(-5000, 5000),  # Force limits (N)
        integral_limits=(-100, 100)   # Anti-windup
    )
    vehicle = SimpleLongitudinalVehicle()
    
    # Storage
    velocities = []
    control_outputs = []
    P_terms = []
    I_terms = []
    D_terms = []
    
    # Simulate
    for t_idx, t in enumerate(time):
        target = target_speed[t_idx]
        current = vehicle.velocity
        
        # PID control
        force, P, I, D = controller.update(target, current)
        
        # Update vehicle
        velocity = vehicle.update(force, dt)
        
        # Record
        velocities.append(velocity)
        control_outputs.append(force)
        P_terms.append(P)
        I_terms.append(I)
        D_terms.append(D)
    
    results[name] = {
        'velocity': np.array(velocities),
        'control': np.array(control_outputs),
        'P': np.array(P_terms),
        'I': np.array(I_terms),
        'D': np.array(D_terms)
    }

# Visualization
fig = plt.figure(figsize=(16, 10))
gs = fig.add_gridspec(3, 2, hspace=0.3, wspace=0.3)

# Plot 1: Velocity tracking
ax1 = fig.add_subplot(gs[0, :])
ax1.plot(time, target_speed, 'k--', linewidth=2, label='Target Speed')
for name, data in results.items():
    ax1.plot(time, data['velocity'], linewidth=2, label=name)
ax1.set_xlabel('Time (s)', fontweight='bold')
ax1.set_ylabel('Velocity (m/s)', fontweight='bold')
ax1.set_title('PID Cruise Control: Velocity Tracking', fontweight='bold', fontsize=14)
ax1.legend(loc='lower right')
ax1.grid(True, alpha=0.3)

# Plot 2 & 3: Control effort for two configurations
for idx, (name, data) in enumerate(list(results.items())[:2]):
    ax = fig.add_subplot(gs[1, idx])
    ax.plot(time, data['control'], 'b-', linewidth=2, label='Total')
    ax.plot(time, data['P'], 'r--', linewidth=1.5, label='P term')
    ax.plot(time, data['I'], 'g--', linewidth=1.5, label='I term')
    ax.plot(time, data['D'], 'm--', linewidth=1.5, label='D term')
    ax.set_xlabel('Time (s)', fontweight='bold')
    ax.set_ylabel('Force (N)', fontweight='bold')
    ax.set_title(f'Control Output: {name}', fontweight='bold')
    ax.legend(loc='upper right', fontsize=9)
    ax.grid(True, alpha=0.3)

# Plot 4 & 5: Error analysis for two configurations
for idx, (name, data) in enumerate(list(results.items())[2:4]):
    ax = fig.add_subplot(gs[2, idx])
    error = target_speed - data['velocity']
    ax.plot(time, error, 'r-', linewidth=2)
    ax.axhline(y=0, color='k', linestyle='--', linewidth=1)
    ax.fill_between(time, 0, error, alpha=0.3, color='red')
    ax.set_xlabel('Time (s)', fontweight='bold')
    ax.set_ylabel('Error (m/s)', fontweight='bold')
    ax.set_title(f'Tracking Error: {name}', fontweight='bold')
    ax.grid(True, alpha=0.3)

plt.suptitle('PID Controller Comparison for Cruise Control', fontsize=16, fontweight='bold')
plt.show()

# Performance metrics
print("=" * 80)
print("PERFORMANCE METRICS")
print("=" * 80)
for name, data in results.items():
    error = target_speed - data['velocity']
    mae = np.mean(np.abs(error))
    rmse = np.sqrt(np.mean(error**2))
    overshoot = np.max(data['velocity'] - target_speed)
    
    # Settling time (within 2% of final value)
    final_target = target_speed[-1]
    tolerance = 0.02 * final_target
    settling_idx = np.where(np.abs(data['velocity'][int(10/dt):] - final_target) > tolerance)[0]
    settling_time = (settling_idx[-1] * dt) if len(settling_idx) > 0 else 0
    
    print(f"\n{name}:")
    print(f"  MAE: {mae:.3f} m/s")
    print(f"  RMSE: {rmse:.3f} m/s")
    print(f"  Max Overshoot: {overshoot:.3f} m/s")
    print(f"  Settling Time: {settling_time:.2f} s")
print("=" * 80)

## 3. Lateral Control (Steering Control)

**Lateral control** manages the vehicle's steering to follow a desired path. Two popular algorithms are **Pure Pursuit** and **Stanley Method**.

---

### 3.1 Pure Pursuit Algorithm

**Pure Pursuit** is a geometric path tracking algorithm that steers the vehicle toward a "lookahead point" on the desired path.

#### Concept

1. Find a **lookahead point** on the path at distance **L_d** ahead
2. Calculate the **arc** connecting the vehicle to the lookahead point
3. Compute **steering angle** to follow this arc

```
        Path
         ╱
        ╱
       ● Lookahead point
      ╱│
     ╱ │ L_d (lookahead distance)
    ╱  │
   ╱   │
  ╱    │
 ╱α    │
●──────┘
Vehicle
```

---

#### Geometry

From the geometry of circular motion:

$$\delta = \arctan\left(\frac{2L \sin(\alpha)}{L_d}\right)$$

Where:
- **δ**: Steering angle
- **L**: Wheelbase
- **α**: Angle between vehicle heading and lookahead point
- **L_d**: Lookahead distance

For small angles: $\delta \approx \frac{2L\alpha}{L_d}$

---

#### Lookahead Distance Selection

**Fixed lookahead**: $L_d = \text{constant}$ (e.g., 5m)
- ✅ Simple
- ❌ Poor performance at different speeds

**Velocity-dependent lookahead**: $L_d = K_v \cdot v + L_{\min}$
- ✅ Better tracking at all speeds
- ✅ Shorter lookahead at low speeds (tighter tracking)
- ✅ Longer lookahead at high speeds (smoother, more stable)

**Typical values**:
- $K_v = 0.5$ to $1.0$ (seconds)
- $L_{\min} = 2$ to $5$ m
- Example: At 20 m/s → $L_d = 0.5 \times 20 + 2 = 12$ m

---

#### Advantages & Disadvantages

**Advantages**:
- ✅ Simple to implement
- ✅ Computationally efficient
- ✅ Smooth control
- ✅ Works well on smooth paths

**Disadvantages**:
- ❌ Large cross-track errors on sharp turns
- ❌ Struggles with discontinuous paths
- ❌ No explicit handling of heading error
- ❌ Sensitive to lookahead distance tuning

---

### 3.2 Stanley Method

The **Stanley method** was developed for the Stanford Racing Team's autonomous vehicle that won the 2005 DARPA Grand Challenge. It improves on Pure Pursuit by explicitly correcting both cross-track and heading errors.

#### Control Law

$$\delta(t) = \psi_e(t) + \arctan\left(\frac{k \cdot e_{fa}(t)}{v(t)}\right)$$

Where:
- **δ**: Steering angle
- **ψ_e**: Heading error (desired heading - actual heading)
- **e_fa**: Cross-track error at front axle
- **k**: Gain parameter (typical: 0.5-5.0)
- **v**: Vehicle speed

---

#### Two Error Terms

**1. Heading Error (ψ_e)**:
- Aligns vehicle heading with path heading
- Dominant at high speeds
- Fast correction

**2. Cross-Track Error (arctan term)**:
- Reduces perpendicular distance to path
- Dominant at low speeds
- Prevents steady-state error

**Softening constant**: To prevent division by zero at v=0:
$$\delta = \psi_e + \arctan\left(\frac{k \cdot e_{fa}}{k_s + v}\right)$$

Where $k_s$ is a small constant (e.g., 1.0 m/s)

---

#### Stanley vs. Pure Pursuit

| Feature | Pure Pursuit | Stanley |
|---------|--------------|---------|
| **Cross-track error** | Implicit (via lookahead) | Explicit correction |
| **Heading error** | Not directly addressed | Explicit correction |
| **Sharp turns** | Cuts corners | Better tracking |
| **Low speeds** | Can oscillate | More stable |
| **Complexity** | Simpler | Slightly more complex |
| **Tuning** | Lookahead distance | Gain k |

---

### 3.3 Model Predictive Control (MPC)

**MPC** (covered in depth in Week 10) is an advanced control technique that optimizes steering over a future horizon.

**Benefits**:
- ✅ Handles constraints (max steering angle, comfort limits)
- ✅ Optimizes trajectory, not just immediate error
- ✅ Can incorporate vehicle dynamics

**Drawbacks**:
- ❌ Computationally expensive
- ❌ Requires accurate vehicle model
- ❌ More complex to tune

---

## 4. Adaptive Cruise Control (ACC)

**Adaptive Cruise Control** extends traditional cruise control by maintaining a safe following distance behind a lead vehicle.

### ACC System Architecture

```
┌─────────────────┐
│  Radar/Camera   │  ← Detect lead vehicle
│  (Sensor)       │
└────────┬────────┘
         │ Distance, Relative Velocity
         ↓
┌─────────────────┐
│  ACC Controller │
│  - Gap control  │
│  - Speed limit  │
└────────┬────────┘
         │ Acceleration Command
         ↓
┌─────────────────┐
│ Longitudinal    │
│ Controller      │
│ (PID/MPC)       │
└────────┬────────┘
         │ Throttle/Brake
         ↓
┌─────────────────┐
│    Actuators    │
└─────────────────┘
```

---

### Control Strategy

**Two operating modes**:

**1. Speed Control Mode** (No lead vehicle)
- Maintain user-set cruise speed
- Simple PID on speed error

**2. Gap Control Mode** (Following lead vehicle)
- Maintain safe time gap
- Control based on relative distance and velocity

---

### Time Gap Control

**Desired gap**: $d_{des} = d_{min} + t_{gap} \cdot v_{ego}$

Where:
- **d_min**: Minimum standstill distance (e.g., 2-5m)
- **t_gap**: Time gap (1.0-2.5 seconds, user-adjustable)
- **v_ego**: Ego vehicle velocity

**Control law**:
$$a = K_1 (d_{actual} - d_{des}) + K_2 (\dot{d}_{actual} - \dot{d}_{des})$$

Where:
- **d_actual**: Actual distance to lead vehicle
- **$\dot{d}_{actual}$**: Relative velocity (closing rate)

---

### Safety Constraints

ACC must respect:
- **Maximum acceleration**: +2.0 to +3.0 m/s²
- **Maximum deceleration**: -3.0 to -4.0 m/s² (comfort)
- **Emergency braking**: Up to -8.0 m/s² (collision imminent)
- **Jerk limits**: ±2.0 m/s³ (rate of acceleration change)

**Time-to-Collision (TTC)**:
$$TTC = \frac{d_{actual}}{\dot{d}_{actual}}$$

If TTC < 2.5 seconds → Apply stronger braking

---

### Cut-in/Cut-out Scenarios

**Cut-in**: Vehicle merges into ego lane ahead
- Detect new lead vehicle
- Quickly adjust speed (higher deceleration allowed)
- Avoid harsh braking (passenger comfort)

**Cut-out**: Lead vehicle changes lanes
- Detect new lead vehicle (farther ahead) or clear road
- Gradually accelerate to cruise speed

---

## 5. Control System Integration

Real autonomous vehicles integrate multiple controllers in a hierarchy:

```
┌─────────────────────────────────────┐
│      Mission Planning               │  ← High-level: Route A→B
└──────────────┬──────────────────────┘
               ↓
┌─────────────────────────────────────┐
│      Behavioral Planning            │  ← Mid-level: Lane change, merge
└──────────────┬──────────────────────┘
               ↓
┌─────────────────────────────────────┐
│      Trajectory Planning            │  ← Low-level: Smooth path
└──────────────┬──────────────────────┘
               ↓
┌──────────────────────┬──────────────┐
│  Lateral Control     │ Longitudinal │  ← Execution: Steering + Speed
│  (Stanley/MPC)       │ Control (PID)│
└──────────────────────┴──────────────┘
               ↓
┌─────────────────────────────────────┐
│         Actuators                   │  ← Hardware: Motors, brakes
│  (Drive-by-Wire)                    │
└─────────────────────────────────────┘
```

**Control loop rates**:
- Planning: 1-10 Hz
- Control: 20-100 Hz
- Actuator feedback: 100-1000 Hz

---

## Exercises

In [ ]:
# Implementation: Pure Pursuit and Stanley Controllers

class PurePursuitController:
    """Pure Pursuit lateral controller."""
    
    def __init__(self, wheelbase, lookahead_gain=0.5, lookahead_min=2.0, lookahead_max=20.0):
        """
        Args:
            wheelbase: Vehicle wheelbase (m)
            lookahead_gain: Velocity-dependent lookahead gain (s)
            lookahead_min: Minimum lookahead distance (m)
            lookahead_max: Maximum lookahead distance (m)
        """
        self.L = wheelbase
        self.k_ld = lookahead_gain
        self.ld_min = lookahead_min
        self.ld_max = lookahead_max
    
    def find_lookahead_point(self, path, vehicle_pos, lookahead_dist):
        """
        Find lookahead point on path.
        
        Args:
            path: Nx2 array of path points [(x, y), ...]
            vehicle_pos: (x, y) current position
            lookahead_dist: Desired lookahead distance
        
        Returns:
            lookahead_point: (x, y) point on path
        """
        # Find closest point on path
        distances = np.linalg.norm(path - vehicle_pos, axis=1)
        closest_idx = np.argmin(distances)
        
        # Search forward along path for point at lookahead distance
        for i in range(closest_idx, len(path)):
            dist = np.linalg.norm(path[i] - vehicle_pos)
            if dist >= lookahead_dist:
                return path[i]
        
        # If no point found, return last point
        return path[-1]
    
    def compute_steering(self, vehicle_state, path):
        """
        Compute steering angle using Pure Pursuit.
        
        Args:
            vehicle_state: [x, y, heading, velocity]
            path: Nx2 array of path points
        
        Returns:
            steering_angle: rad
        """
        x, y, heading, v = vehicle_state
        vehicle_pos = np.array([x, y])
        
        # Calculate lookahead distance (velocity-dependent)
        lookahead_dist = np.clip(self.k_ld * v + self.ld_min, 
                                  self.ld_min, self.ld_max)
        
        # Find lookahead point
        lookahead_point = self.find_lookahead_point(path, vehicle_pos, lookahead_dist)
        
        # Calculate alpha (angle between vehicle heading and lookahead point)
        dx = lookahead_point[0] - x
        dy = lookahead_point[1] - y
        angle_to_goal = np.arctan2(dy, dx)
        alpha = angle_to_goal - heading
        
        # Normalize alpha to [-pi, pi]
        alpha = np.arctan2(np.sin(alpha), np.cos(alpha))
        
        # Calculate steering angle using Pure Pursuit formula
        steering_angle = np.arctan2(2.0 * self.L * np.sin(alpha), lookahead_dist)
        
        return steering_angle


class StanleyController:
    """Stanley lateral controller."""
    
    def __init__(self, wheelbase, k=1.0, k_soft=1.0):
        """
        Args:
            wheelbase: Vehicle wheelbase (m)
            k: Cross-track error gain
            k_soft: Softening constant for low speeds (m/s)
        """
        self.L = wheelbase
        self.k = k
        self.k_soft = k_soft
    
    def compute_steering(self, vehicle_state, path):
        """
        Compute steering angle using Stanley method.
        
        Args:
            vehicle_state: [x, y, heading, velocity]
            path: Nx2 array of path points
        
        Returns:
            steering_angle: rad
        """
        x, y, heading, v = vehicle_state
        
        # Find closest point on path
        distances = np.linalg.norm(path - np.array([x, y]), axis=1)
        closest_idx = np.argmin(distances)
        
        if closest_idx >= len(path) - 1:
            closest_idx = len(path) - 2
        
        # Get path segment
        p1 = path[closest_idx]
        p2 = path[closest_idx + 1]
        
        # Path heading at closest point
        path_heading = np.arctan2(p2[1] - p1[1], p2[0] - p1[0])
        
        # Heading error
        heading_error = path_heading - heading
        heading_error = np.arctan2(np.sin(heading_error), np.cos(heading_error))
        
        # Cross-track error at front axle
        # Front axle position
        fx = x + self.L * np.cos(heading)
        fy = y + self.L * np.sin(heading)
        
        # Vector from front axle to closest point
        dx = p1[0] - fx
        dy = p1[1] - fy
        
        # Project onto path normal
        cross_track_error = -dx * np.sin(path_heading) + dy * np.cos(path_heading)
        
        # Stanley control law
        steering_angle = heading_error + np.arctan2(self.k * cross_track_error, 
                                                      self.k_soft + v)
        
        return steering_angle


# Kinematic Bicycle Model (from Week 2)
class KinematicBicycleModel:
    """
    Implements the kinematic bicycle model for vehicle motion.
    
    State: [x, y, psi, v]
    Control: [delta, a]
    """
    
    def __init__(self, wheelbase=2.5, max_steer=np.deg2rad(30), dt=0.1):
        """
        Args:
            wheelbase: Distance between front and rear axles (m)
            max_steer: Maximum steering angle (rad)
            dt: Time step for simulation (s)
        """
        self.L = wheelbase
        self.max_steer = max_steer
        self.dt = dt
        
    def update(self, state, control):
        """
        Update state using kinematic bicycle model equations.
        
        Args:
            state: [x, y, psi, v] - current state
            control: [delta, a] - steering angle (rad), acceleration (m/s²)
            
        Returns:
            new_state: [x, y, psi, v] - updated state
        """
        x, y, psi, v = state
        delta, a = control
        
        # Clamp steering angle
        delta = np.clip(delta, -self.max_steer, self.max_steer)
        
        # Kinematic bicycle model equations
        dx = v * np.cos(psi)
        dy = v * np.sin(psi)
        dpsi = (v / self.L) * np.tan(delta)
        dv = a
        
        # Euler integration
        new_x = x + dx * self.dt
        new_y = y + dy * self.dt
        new_psi = psi + dpsi * self.dt
        new_v = v + dv * self.dt
        
        # Keep velocity non-negative
        new_v = max(0, new_v)
        
        return np.array([new_x, new_y, new_psi, new_v])
    
    def simulate_trajectory(self, initial_state, control_sequence):
        """
        Simulate vehicle trajectory given control sequence.
        
        Args:
            initial_state: [x, y, psi, v] - starting state
            control_sequence: List of [delta, a] controls
            
        Returns:
            trajectory: Array of states [N x 4]
        """
        trajectory = [initial_state]
        state = initial_state
        
        for control in control_sequence:
            state = self.update(state, control)
            trajectory.append(state.copy())
            
        return np.array(trajectory)


# Integrated Path Tracking Simulation

# Generate a curved path (sine wave)
path_x = np.linspace(0, 100, 500)
path_y = 10 * np.sin(path_x / 15) + 5
reference_path = np.column_stack([path_x, path_y])

# Initial vehicle state
initial_state = np.array([0.0, 0.0, 0.0, 10.0])  # x, y, heading, velocity

# Create controllers
pure_pursuit = PurePursuitController(wheelbase=2.5, lookahead_gain=0.5, lookahead_min=3.0)
stanley = StanleyController(wheelbase=2.5, k=1.0, k_soft=1.0)

# Create vehicle model
vehicle_model = KinematicBicycleModel(wheelbase=2.5, dt=0.1)

# Simulation
dt = 0.1
duration = 15.0
steps = int(duration / dt)

results = {}

for controller_name, controller in [('Pure Pursuit', pure_pursuit), ('Stanley', stanley)]:
    state = initial_state.copy()
    trajectory = [state.copy()]
    steering_angles = []
    cross_track_errors = []
    
    for step in range(steps):
        # Compute steering
        steering = controller.compute_steering(state, reference_path)
        steering_angles.append(steering)
        
        # Update vehicle (constant speed)
        control = [steering, 0.0]  # No acceleration
        state = vehicle_model.update(state, control)
        trajectory.append(state.copy())
        
        # Compute cross-track error
        vehicle_pos = state[:2]
        distances = np.linalg.norm(reference_path - vehicle_pos, axis=1)
        cte = np.min(distances)
        cross_track_errors.append(cte)
    
    results[controller_name] = {
        'trajectory': np.array(trajectory),
        'steering': np.array(steering_angles),
        'cte': np.array(cross_track_errors)
    }

# Visualization
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

colors = {'Pure Pursuit': '#3498db', 'Stanley': '#e74c3c'}

# Plot 1: Trajectories
ax1 = axes[0, 0]
ax1.plot(reference_path[:, 0], reference_path[:, 1], 'k--', linewidth=2, label='Reference Path')
for name, data in results.items():
    traj = data['trajectory']
    ax1.plot(traj[:, 0], traj[:, 1], linewidth=2, label=name, color=colors[name])
    ax1.plot(traj[0, 0], traj[0, 1], 'o', markersize=10, color=colors[name])
    ax1.plot(traj[-1, 0], traj[-1, 1], 's', markersize=10, color=colors[name])

ax1.set_xlabel('X Position (m)', fontweight='bold')
ax1.set_ylabel('Y Position (m)', fontweight='bold')
ax1.set_title('Path Tracking Comparison', fontweight='bold', fontsize=13)
ax1.legend()
ax1.grid(True, alpha=0.3)
ax1.axis('equal')

# Plot 2: Cross-track error
ax2 = axes[0, 1]
time = np.arange(steps) * dt
for name, data in results.items():
    ax2.plot(time, data['cte'], linewidth=2, label=name, color=colors[name])
ax2.set_xlabel('Time (s)', fontweight='bold')
ax2.set_ylabel('Cross-Track Error (m)', fontweight='bold')
ax2.set_title('Tracking Accuracy', fontweight='bold', fontsize=13)
ax2.legend()
ax2.grid(True, alpha=0.3)

# Plot 3: Steering angles
ax3 = axes[1, 0]
for name, data in results.items():
    ax3.plot(time, np.rad2deg(data['steering']), linewidth=2, label=name, color=colors[name])
ax3.set_xlabel('Time (s)', fontweight='bold')
ax3.set_ylabel('Steering Angle (deg)', fontweight='bold')
ax3.set_title('Control Commands', fontweight='bold', fontsize=13)
ax3.legend()
ax3.grid(True, alpha=0.3)

# Plot 4: Metrics comparison
ax4 = axes[1, 1]
metrics_names = ['Mean CTE', 'Max CTE', 'Mean |Steering|']
pure_pursuit_metrics = [
    np.mean(results['Pure Pursuit']['cte']),
    np.max(results['Pure Pursuit']['cte']),
    np.mean(np.abs(results['Pure Pursuit']['steering']))
]
stanley_metrics = [
    np.mean(results['Stanley']['cte']),
    np.max(results['Stanley']['cte']),
    np.mean(np.abs(results['Stanley']['steering']))
]

x = np.arange(len(metrics_names))
width = 0.35

bars1 = ax4.bar(x - width/2, pure_pursuit_metrics, width, label='Pure Pursuit', 
                color=colors['Pure Pursuit'], alpha=0.8)
bars2 = ax4.bar(x + width/2, stanley_metrics, width, label='Stanley', 
                color=colors['Stanley'], alpha=0.8)

ax4.set_ylabel('Value', fontweight='bold')
ax4.set_title('Performance Metrics Comparison', fontweight='bold', fontsize=13)
ax4.set_xticks(x)
ax4.set_xticklabels(metrics_names)
ax4.legend()
ax4.grid(True, alpha=0.3, axis='y')

# Add value labels on bars
for bars in [bars1, bars2]:
    for bar in bars:
        height = bar.get_height()
        ax4.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.3f}', ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.suptitle('Lateral Control: Pure Pursuit vs. Stanley Method', 
             fontsize=16, fontweight='bold', y=1.00)
plt.show()

# Print detailed metrics
print("=" * 80)
print("LATERAL CONTROL PERFORMANCE COMPARISON")
print("=" * 80)
for name, data in results.items():
    cte = data['cte']
    steering = data['steering']
    
    print(f"\n{name}:")
    print(f"  Mean cross-track error: {np.mean(cte):.4f} m")
    print(f"  Max cross-track error: {np.max(cte):.4f} m")
    print(f"  RMS cross-track error: {np.sqrt(np.mean(cte**2)):.4f} m")
    print(f"  Mean |steering angle|: {np.rad2deg(np.mean(np.abs(steering))):.2f}°")
    print(f"  Max |steering angle|: {np.rad2deg(np.max(np.abs(steering))):.2f}°")
    print(f"  Steering smoothness (std): {np.rad2deg(np.std(steering)):.2f}°")
print("=" * 80)

---

## References

### Control Theory Fundamentals

1. **Åström, K. J., & Murray, R. M.** (2021). *Feedback Systems: An Introduction for Scientists and Engineers* (2nd ed.). Princeton University Press.
   - Comprehensive introduction to control theory, including PID control

2. **Franklin, G. F., Powell, J. D., & Emami-Naeini, A.** (2019). *Feedback Control of Dynamic Systems* (8th ed.). Pearson.
   - Classic textbook on control systems

3. **Ogata, K.** (2010). *Modern Control Engineering* (5th ed.). Prentice Hall.
   - Detailed treatment of PID controllers and tuning methods

### Drive-by-Wire Systems

4. **ISO 26262** (2018). "Road vehicles — Functional safety." *International Organization for Standardization*.
   - Safety standard for automotive systems including drive-by-wire

5. **Isermann, R., Schwarz, R., & Stölzl, S.** (2002). "Fault-tolerant drive-by-wire systems." *IEEE Control Systems Magazine, 22*(5), 64-81.
   - Overview of drive-by-wire architecture and safety

6. **Ploeg, J., Shukla, D. P., van de Wouw, N., & Nijmeijer, H.** (2014). "Controller synthesis for string stability of vehicle platoons." *IEEE Transactions on Intelligent Transportation Systems, 15*(2), 854-865.

### PID Control for Vehicles

7. **Rajamani, R.** (2011). *Vehicle Dynamics and Control* (2nd ed.). Springer.
   - Chapter 6: Longitudinal vehicle dynamics and control

8. **Kiencke, U., & Nielsen, L.** (2005). *Automotive Control Systems: For Engine, Driveline, and Vehicle* (2nd ed.). Springer.
   - Practical implementation of automotive control systems

### Lateral Control Algorithms

9. **Snider, J. M.** (2009). "Automatic Steering Methods for Autonomous Automobile Path Tracking." *Robotics Institute, Carnegie Mellon University*, CMU-RI-TR-09-08.
   - Comprehensive comparison of Pure Pursuit, Stanley, and MPC

10. **Coulter, R. C.** (1992). "Implementation of the Pure Pursuit Path Tracking Algorithm." *Robotics Institute, Carnegie Mellon University*, CMU-RI-TR-92-01.
    - Original Pure Pursuit algorithm

11. **Thrun, S., et al.** (2006). "Stanley: The Robot That Won the DARPA Grand Challenge." *Journal of Field Robotics, 23*(9), 661-692.
    - Description of Stanley method from DARPA Grand Challenge winner

12. **Hoffman, G. M., Tomlin, C. J., Montemerlo, M., & Thrun, S.** (2008). "Autonomous Automobile Trajectory Tracking for Off-Road Driving: Controller Design, Experimental Validation and Racing." *American Control Conference*.

### Adaptive Cruise Control

13. **Milanes, V., & Shladover, S. E.** (2014). "Modeling cooperative and autonomous adaptive cruise control dynamic responses using experimental data." *Transportation Research Part C: Emerging Technologies, 48*, 285-300.

14. **Swaroop, D., & Hedrick, J. K.** (1996). "String stability of interconnected systems." *IEEE Transactions on Automatic Control, 41*(3), 349-357.
    - Theory behind vehicle following systems

15. **Ioannou, P. A., & Chien, C. C.** (1993). "Autonomous intelligent cruise control." *IEEE Transactions on Vehicular Technology, 42*(4), 657-672.

### Model Predictive Control (Preview)

16. **Falcone, P., Borrelli, F., Asgari, J., Tseng, H. E., & Hrovat, D.** (2007). "Predictive active steering control for autonomous vehicle systems." *IEEE Transactions on Control Systems Technology, 15*(3), 566-580.

17. **Borrelli, F., Bemporad, A., & Morari, M.** (2017). *Predictive Control for Linear and Hybrid Systems*. Cambridge University Press.

### Implementation and Practice

18. **Paden, B., Čáp, M., Yong, S. Z., Yershov, D., & Frazzoli, E.** (2016). "A survey of motion planning and control techniques for self-driving urban vehicles." *IEEE Transactions on Intelligent Vehicles, 1*(1), 33-55.

19. **Polack, P., et al.** (2017). "The Kinematic Bicycle Model: A Consistent Model for Planning Feasible Trajectories for Autonomous Vehicles?" *IEEE Intelligent Vehicles Symposium*.

20. **Kong, J., et al.** (2015). "Kinematic and Dynamic Vehicle Models for Autonomous Driving Control Design." *IEEE Intelligent Vehicles Symposium*.

### Software and Tools

21. **Python Control Systems Library**: https://python-control.readthedocs.io/
    - Python library for control system analysis and design

22. **MATLAB/Simulink**: Automated Driving Toolbox
    - Industry-standard tools for vehicle control design

23. **Baidu Apollo Planning Module**: https://github.com/ApolloAuto/apollo/tree/master/modules/control
    - Open-source implementation of vehicle controllers

### Online Resources

24. **Brian Douglas Control Systems Lectures**: https://www.youtube.com/user/ControlLectures
    - Excellent video tutorials on control theory

25. **MIT OpenCourseWare - Feedback Control Systems**: https://ocw.mit.edu/courses/mechanical-engineering/

---

## Summary

In this notebook, you learned:

1. **Drive-by-Wire Systems**: Electronic control of steering, throttle, and braking
   - Architecture and safety requirements (ASIL-D)
   - Redundancy and fault tolerance
   - CAN bus communication

2. **Longitudinal Control**: Managing vehicle speed
   - PID controller fundamentals (P, I, D terms)
   - Tuning methods (Ziegler-Nichols, manual)
   - Anti-windup and derivative kick prevention

3. **Lateral Control**: Steering to follow paths
   - Pure Pursuit algorithm (geometric, lookahead-based)
   - Stanley method (heading + cross-track error)
   - Comparison and trade-offs

4. **Adaptive Cruise Control**: Following lead vehicles
   - Time gap control
   - Safety constraints (TTC, acceleration limits)
   - Cut-in/cut-out scenarios

5. **Integration**: Hierarchical control architecture
   - Planning → Control → Actuation
   - Loop rates and timing

These controllers form the foundation for autonomous vehicle motion execution. You'll build on this in later weeks with advanced techniques like Model Predictive Control (Week 10) and behavioral planning (Week 8).

---

**Next Steps**: Proceed to [Week 4: Sensor Technologies & Fusion](week04_sensor_technologies_fusion.ipynb)